# Build a conversational language understanding model

The Azure AI Language conversational language understanding service (CLU) enables you to train a model that apps can use to extract meaning from natural language.

- Provision Azure resources for Azure AI Language resource
- Define intents, utterances, and entities
- Use patterns to differentiate similar utterances
- Use pre-built entity components
- Train, test, publish, and review an Azure AI Language model

# Importing libraries

In [49]:
from dotenv import load_dotenv
import os
import json
from datetime import date

In [50]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.language.conversations import ConversationAnalysisClient

In [51]:
from functools import lru_cache

# Loading configurations

In [52]:
load_dotenv()
ai_endpoint = os.getenv('AI_LANGUAGE_SERVICE_ENDPOINT')
ai_key = os.getenv('AI_LANGUAGE_SERVICE_KEY')
project_name = os.getenv('PROJECT_NAME')
deployment_name = os.getenv('DEPLOYMENT_NAME')

True

# Calling API [analyze_conversation()]

In [53]:
@lru_cache(maxsize=None)
def analyze_conversation(query_text):
  client = ConversationAnalysisClient(ai_endpoint, AzureKeyCredential(ai_key))

  with client:
    result = client.analyze_conversation(
        task={
            "kind": "Conversation",
            "analysisInput": {
                "conversationItem": {
                    "participantId": "1",
                    "id": "1",
                    "modality": "text",
                    "language": "en",
                    "text": query_text
                },
                "isLoggingEnabled": False
            },
            "parameters": {
                "projectName": project_name,
                "deploymentName": deployment_name,
                "verbose": True
            }
        }
    )

  return result

In [54]:
print(json.dumps(analyze_conversation("Hi what's the current time?"), indent=2))

{
  "kind": "ConversationResult",
  "result": {
    "query": "Hi what's the current time?",
    "prediction": {
      "topIntent": "GetTime",
      "projectKind": "Conversation",
      "intents": [
        {
          "category": "GetTime",
          "confidenceScore": 0.9576142
        },
        {
          "category": "GetDate",
          "confidenceScore": 0.9167327
        },
        {
          "category": "GetDay",
          "confidenceScore": 0.8541195
        },
        {
          "category": "None",
          "confidenceScore": 0
        }
      ],
      "entities": []
    }
  }
}


# Handling user intents

In [55]:
def handle_get_time(entities):
  location = 'local'
  if entities:
    for entity in entities:
      if entity.get("category") == 'Location':
        location = entity.get("text")

  print(f"You asked for the time in: {location}")
  print()


def handle_get_day(entities):
  date_string = date.today().strftime("%m/%d/%Y")
  if entities:
    for entity in entities:
      if entity.get("category") == 'Date':
        date_string = entity.get("text")

  print(f"You asked for the day on: {date_string}")
  print()


def handle_get_date(entities):
  day = 'today'
  if entities:
    for entity in entities:
      if entity.get("category") == 'Weekday':
        day = entity.get("text")

  print(f"You asked for the date on: {day}")
  print()

In [56]:
intent_handlers = {
    'GetTime': handle_get_time,
    'GetDay': handle_get_day,
    'GetDate': handle_get_date
}


def handle_intent(top_intent, entities):
  handler = intent_handlers.get(top_intent)
  if handler:
    handler(entities)
  else:
    print('Try asking me for the time, the day, or the date.')

In [57]:
def analyze_and_handle_intent(query):
  print(f"Query: {query}")

  result = analyze_conversation(query)
  top_intent = result["result"]["prediction"]["topIntent"]
  confidence = result["result"]["prediction"]["intents"][0]["confidenceScore"]
  entities = result["result"]["prediction"]["entities"]

  print(f"Top Intent: {top_intent} (Confidence: {confidence})")
  print("Entities:")
  for entity in entities:
    print(f"  - Category: {entity.get('category')
                           }, Text: {entity.get('text')}")

  print()
  handle_intent(top_intent, entities)

# Main logic

In [58]:
handle_get_time([{'category': 'Location', 'text': 'London'}])

You asked for the time in: London



In [59]:
analyze_and_handle_intent(
    "Can you tell me what's the day in London in 26 January 2024?")

Query: Can you tell me what's the day in London in 26 January 2024?


Top Intent: GetDay (Confidence: 0.8828023)
Entities:
  - Category: Location, Text: London
  - Category: Date, Text: 26 January 2024

You asked for the day on: 26 January 2024



In [60]:
analyze_and_handle_intent(
    "Can you tell me what was the date on last Wednesday in Delhi?")

Query: Can you tell me what was the date on last Wednesday in Delhi?
Top Intent: GetDate (Confidence: 0.91121244)
Entities:
  - Category: Date, Text: last Wednesday
  - Category: Weekday, Text: Wednesday
  - Category: Location, Text: Delhi

You asked for the date on: Wednesday

